#Two-Stages Deep Learning Project
<p>This project has been developped by the Montreal Hacknight community.<p>

<p><ul>https://github.com/robintibor/braindecode</ul>
<ul>https://github.com/NeuroTechX/moabb</ul></p>

<p>This project wouldn't have been possible without the contribution of these people: Arna (...), Justin (...), Yannick Roy, Eamon Egan, (...) and it is currently being supported by Fred Simard (fs@re-ak.com)</p>

<p>This project consist in deriving a two-stages training process from the brain decode implementation of a Shallow Convolutional Network. The core motivation is to combine the strenghts of deep learning - that is being one of the most powerful ML algorithms out-there - while grinding down it's fall back - it requires an immense of data.</p>

<p>The goal is to develop a method to train the network in two stages:</p>

<p><ul>1) Train the network over a large dataset, this training forms some kind of prior over the network (all but one subject)</ul>
<ul>2) Train the network a second time, on the dataset of interest (the subject left aside)</ul></p>

<p>The premise motivating this approach is that the second training is likely to proceed faster than what is normally required to train a deep net, without sacrificing the performance, and will enable the possibility to train a deep learner on a smaller dataset; further, we suspect that the second training will be able to exploit the specificities of the dataset of interest in a way to surpasses the first training, which was trained on a generalistic dataset.</p>

In [3]:


"""
#
# Download the dataset and segment per subject, 
# NOTE: this block should be executed once and once only
#
"""
import os
if not os.path.exists("bnci-2014"):
  os.mkdir("bnci-2014")

#if not os.path.exists("bnci-2014/S01E.mat"):
if True:
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14E.mat -P "bnci-2014"
  
  
#note just in case, there are duplicates
!rm bnci-2014/*.mat.*

--2018-12-21 02:15:14--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01E.mat [following]
--2018-12-21 02:15:14--  https://lampx.tugraz.at/~bci/database/002-2014/S01E.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23769588 (23M)
Saving to: ‘bnci-2014/S01E.mat.1’

S01E.mat.1          100%[===================>]  22.67M  6.55MB/s    in 6.5s    

2018-12-21 02:15:21 (3.47 MB/s) - ‘bnci-2014/S01E.mat.1’ saved [23769588/23769588]

--2018-12-21 02:15:22--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci

In [0]:
# Installation
# NOTE: Should only be called once

#install brain decode
!pip install braindecode -q

# install pytorch
# ref: http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
#  cu91/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
import torch



In [5]:

import numpy as np
import scipy.io as sio

DATASET_PATH = "bnci-2014"
TEST_SUBJECT_IDX = 0 # needs be
TEST_SUBJECT_IDX *= 2

TRIAL_WINDOW_LENGTH = 7

allSubjects = os.listdir(DATASET_PATH)
allSubjects.sort()
testSubject = [allSubjects[TEST_SUBJECT_IDX], allSubjects[TEST_SUBJECT_IDX+1]]
# double pop to remove *T and *E
allSubjects.pop(0)
allSubjects.pop(0) 

print("Test subject: " + repr(testSubject))
print("Train subjects: " + repr(allSubjects))

X1 = []
y1 = []


for f in allSubjects:
    # read file
    d1T = sio.loadmat(DATASET_PATH + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = TRIAL_WINDOW_LENGTH*samplingRate

    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y1.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X1.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X1 = np.array(X1).astype(np.float32)
y1 = (np.array(y1).flatten()-1).astype(np.int64)

# rand permute dataset
idx = np.random.permutation(X1.shape[0])
X1 = X1[idx,:,:]
y1 = y1[idx]


X2 = []
y2 = []

for f in testSubject:
    # read file
    d1T = sio.loadmat(DATASET_PATH + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = TRIAL_WINDOW_LENGTH*samplingRate

    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y2.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X2.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

X2 = np.array(X2).astype(np.float32)
y2 = (np.array(y2).flatten()-1).astype(np.int64)


# rand permute dataset
idx = np.random.permutation(X2.shape[0])

X2 = X2[idx,:,:]
y2 = y2[idx]

Test subject: ['S01E.mat', 'S01T.mat']
Train subjects: ['S02E.mat', 'S02T.mat', 'S03E.mat', 'S03T.mat', 'S04E.mat', 'S04T.mat', 'S05E.mat', 'S05T.mat', 'S06E.mat', 'S06T.mat', 'S07E.mat', 'S07T.mat', 'S08E.mat', 'S08T.mat', 'S09E.mat', 'S09T.mat', 'S10E.mat', 'S10T.mat', 'S11E.mat', 'S11T.mat', 'S12E.mat', 'S12T.mat', 'S13E.mat', 'S13T.mat', 'S14E.mat', 'S14T.mat']


In [6]:

# if dl-eeg-playground already here, erase
!rm -rf dl-eeg-playground
!rm -rf brainDecodeSKLearnWrapper

!git clone https://github.com/NeuroTechX/dl-eeg-playground.git 
!mv dl-eeg-playground/brainDecode/towardMoabbIntegration/brainDecodeSKLearnWrapper .

Cloning into 'dl-eeg-playground'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 294 (delta 17), reused 23 (delta 9), pack-reused 261
Receiving objects: 100% (294/294), 2.17 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [11]:
import sys
sys.path.append("brainDecode/towardMoabbIntegration")

# load the general network Trainer
from brainDecodeSKLearnWrapper.ShallowFBCSPNet_GeneralTrainer import ShallowFBCSPNet_GeneralTrainer



classifier = ShallowFBCSPNet_GeneralTrainer(nb_epoch=100)
classifier.fit(X1,y1)


Epoch 0
Train  Loss: 0.88101
Train  Accuracy: 52.5%
Test   Loss: 0.86185
Test   Accuracy: 51.5%
Epoch 1
Train  Loss: 1.15238
Train  Accuracy: 50.3%
Test   Loss: 1.23255
Test   Accuracy: 50.4%
Epoch 2
Train  Loss: 0.70361
Train  Accuracy: 55.1%
Test   Loss: 0.72213
Test   Accuracy: 53.8%
Epoch 3
Train  Loss: 0.70020
Train  Accuracy: 55.5%
Test   Loss: 0.73845
Test   Accuracy: 57.3%
Epoch 4
Train  Loss: 0.67466
Train  Accuracy: 58.5%
Test   Loss: 0.72315
Test   Accuracy: 55.4%
Epoch 5
Train  Loss: 0.72816
Train  Accuracy: 53.9%
Test   Loss: 0.77664
Test   Accuracy: 52.7%
Epoch 6
Train  Loss: 0.72045
Train  Accuracy: 54.2%
Test   Loss: 0.75981
Test   Accuracy: 51.5%
Epoch 7
Train  Loss: 0.90019
Train  Accuracy: 50.3%
Test   Loss: 0.91848
Test   Accuracy: 49.2%
Epoch 8
Train  Loss: 1.47195
Train  Accuracy: 50.2%
Test   Loss: 1.48199
Test   Accuracy: 49.2%
Epoch 9
Train  Loss: 0.98835
Train  Accuracy: 50.1%
Test   Loss: 1.07659
Test   Accuracy: 50.8%
Epoch 10
Train  Loss: 0.66306
Train  Acc

ShallowFBCSPNet_GeneralTrainer(filter_time_length=75, n_filters_spat=5,
                n_filters_time=10, nb_epoch=100, pool_time_length=60,
                pool_time_stride=30)

In [12]:
from torch import optim
from torch import nn
from braindecode.datautil.signal_target import SignalAndTarget

# define a number of train/test trials
classifier.model.conv_classifier = nn.Conv2d(5, 2,(116, 1), bias=True).cuda()

# define a number of train/test trials
nb_train_trials = int(np.floor(7/8*X2.shape[0]))

# split the dataset
train_set = SignalAndTarget(X2[:nb_train_trials], y=y2[:nb_train_trials])
test_set = SignalAndTarget(X2[nb_train_trials:], y=y2[nb_train_trials:])

classifier.optimizer = optim.Adam(classifier.model.parameters())

nbIter = 50

# array that tracks results
classifier.loss_rec = np.zeros((nbIter,2))
classifier.accuracy_rec = np.zeros((nbIter,2))

# run all epoch
for i_epoch in range(nbIter):

    classifier._batchTrain(i_epoch, train_set)
    classifier._evalTraining(i_epoch, train_set, test_set)

Epoch 0
Train  Loss: 0.67469
Train  Accuracy: 55.0%
Test   Loss: 0.75461
Test   Accuracy: 45.0%
Epoch 1
Train  Loss: 0.63450
Train  Accuracy: 62.1%
Test   Loss: 0.72401
Test   Accuracy: 45.0%
Epoch 2
Train  Loss: 0.57586
Train  Accuracy: 74.3%
Test   Loss: 0.73132
Test   Accuracy: 50.0%
Epoch 3
Train  Loss: 0.53469
Train  Accuracy: 79.3%
Test   Loss: 0.73820
Test   Accuracy: 55.0%
Epoch 4
Train  Loss: 0.49852
Train  Accuracy: 82.9%
Test   Loss: 0.74655
Test   Accuracy: 60.0%
Epoch 5
Train  Loss: 0.46743
Train  Accuracy: 87.1%
Test   Loss: 0.75755
Test   Accuracy: 55.0%
Epoch 6
Train  Loss: 0.43757
Train  Accuracy: 90.0%
Test   Loss: 0.77154
Test   Accuracy: 50.0%
Epoch 7
Train  Loss: 0.40752
Train  Accuracy: 92.1%
Test   Loss: 0.77195
Test   Accuracy: 50.0%
Epoch 8
Train  Loss: 0.38145
Train  Accuracy: 93.6%
Test   Loss: 0.76254
Test   Accuracy: 50.0%
Epoch 9
Train  Loss: 0.35862
Train  Accuracy: 92.9%
Test   Loss: 0.75538
Test   Accuracy: 55.0%
Epoch 10
Train  Loss: 0.33759
Train  Acc